### Straight Assembly Line Optimization

The objective of this optimization problem is to minimize the number of open stations while ensuring that tasks are assigned to stations such that the cycle time is not exceeded and the precedence constraints are met.

#### Variables:
- $x_{ik}$: Binary variable, 1 if task $i$ is assigned to station $k$, 0 otherwise.
- $z_k$: Binary variable, 1 if station $k$ is open, 0 otherwise.

#### Objective Function:
Minimize the number of open stations:
$$ \text{Minimize} \sum_{k=1}^{K} z_k $$

#### Constraints:

1. **Task Assignment Constraint:** Each task must be assigned to exactly one station.
   $$ \sum_{k=1}^{K} x_{ik} = 1 \quad \forall i $$

2. **Cycle Time Constraint:** The sum of the task times at each station must not exceed the cycle time for that station.
   $$ \sum_{i=1}^{n} t_i \cdot x_{ik} \leq C \cdot z_k \quad \forall k $$

3. **Precedence Constraint:** Tasks must respect the given precedence relationships, where task $j$ can only start after task $i$ if specified.
   $$ x_{ik} \leq x_{jk} \quad \forall (i, j) \in \text{Precedence}, \forall k $$

Where:
- $i$ and $j$ are indices for tasks,
- $k$ is an index for stations,
- $t_i$ is the time required to complete task $i$,
- $C$ is the cycle time limit for each station,
- $K$ is the total number of stations,
- $\text{Precedence}$ is the set of ordered pairs of tasks $(i, j)$ where task $i$ must precede task $j$.


In [1]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary


tasks = {1: 19, 2: 12, 3: 12, 4: 14, 5: 20, 6: 14, 7: 22, 8: 13, 9: 15, 10: 30, 11: 12, 12: 13, 13: 13, 14: 15, 15: 9, 16: 21, 17: 16, 18: 12, 19: 18, 20: 19, 21: 21, 22: 21, 23: 8}

# Define the maximum number of stations
max_stations = 6


cycle_time = 70

precedence = [(1, 2), (2, 3), (3, 4)] # ... and so on.

# Initialize the problem
model = LpProblem("Assembly_Line_Balancing", LpMinimize)

# Variables for task assignment to stations (x) and station activation (z)
x = LpVariable.dicts("task_assignment", [(i, k) for i in tasks for k in range(1, max_stations + 1)], cat=LpBinary)
z = LpVariable.dicts("station_open", range(1, max_stations + 1), cat=LpBinary)

# Objective function: Minimize the number of open stations
model += lpSum(z[k] for k in range(1, max_stations + 1))

# Constraints

# Each task must be assigned to exactly one station
for i in tasks:
    model += lpSum(x[(i, k)] for k in range(1, max_stations + 1)) == 1

# Sum of task times at each station must not exceed the cycle time for that station
for k in range(1, max_stations + 1):
    model += lpSum(tasks[i] * x[(i, k)] for i in tasks) <= cycle_time * z[k]

# Precedence constraints
for (i, j) in precedence:
    for k in range(1, max_stations + 1):
        model += x[(i, k)] <= x[(j, k)]

# Solve the model
model.solve()

# Output the results
for k in range(1, max_stations + 1):
    if z[k].value() == 1:
        print(f'Station {k} is open')
        for i in tasks:
            if x[(i, k)].value() == 1:
                print(f' - Task {i} assigned to station {k}')


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/OR/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/c3e4e6bff32f43268043d19097a8d7d8-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/c3e4e6bff32f43268043d19097a8d7d8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 52 COLUMNS
At line 665 RHS
At line 713 BOUNDS
At line 858 ENDATA
Problem MODEL has 47 rows, 144 columns and 318 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5.27143 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 76 substitutions
Cgl0004I processed model has 26 rows, 126 columns (126 integer (126 of which binary)) and 246 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 11 int